In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import seaborn as sns
import sklearn.linear_model as sk_lm
from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import confusion_matrix, classification_report, precision_score
import random
from sklearn.utils import shuffle
from sklearn.model_selection  import train_test_split
import os
import glob
#import 
%matplotlib inline

Source: https://www.kaggle.com/grouplens/movielens-20m-dataset/version/2


Context

The datasets describe ratings and free-text tagging activities from MovieLens, a movie recommendation service. It contains 20000263 ratings and 465564 tag applications across 27278 movies. These data were created by 138493 users between January 09, 1995 and March 31, 2015. This dataset was generated on October 17, 2016.

Users were selected at random for inclusion. All selected users had rated at least 20 movies.
Content

No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in six files.

tag.csv that contains tags applied to movies by users:

    userId

    movieId

    tag

    timestamp

rating.csv that contains ratings of movies by users:

    userId

    movieId

    rating

    timestamp

movie.csv that contains movie information:

    movieId

    title

    genres

link.csv that contains identifiers that can be used to link to other sources:

    movieId

    imdbId

    tmbdId

genome_scores.csv that contains movie-tag relevance data:

    movieId

    tagId

    relevance

genome_tags.csv that contains tag descriptions:

    tagId

    tag

Acknowledgements

The original datasets can be found here. To acknowledge use of the dataset in publications, please cite the following paper:

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872
Inspiration

Some ideas worth exploring:

    Which genres receive the highest ratings? How does this change over time?

    Determine the temporal trends in the genres/tagging activity of the movies released


In [2]:
x = glob.glob("/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/*.csv")
x

['/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/genome_scores.csv',
 '/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/genome_tags.csv',
 '/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/link.csv',
 '/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/movie.csv',
 '/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/rating.csv',
 '/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/tag.csv']

In [3]:
y=[]
for i in x:
    y.append(i.strip("/media/radial/Work/Work/Work on datasets/movielens-20m-dataset").strip(".c"))
y = [x[2:] for x in y]
y

['genome_scores', 'genome_tags', 'link', 'movie', 'rating', 'tag']

In [4]:
genome_scores = pd.read_csv(x[0])
genome_tags = pd.read_csv(x[1])
link = pd.read_csv(x[2])
movie = pd.read_csv(x[3])
rating = pd.read_csv(x[4])
tag = pd.read_csv(x[5])

In [10]:
print("genome_scores............................................................................................")
print(genome_scores.head())
print("genome_tags............................................................................................")
print(genome_tags.head())
print("link............................................................................................")
print(link.head())
print("movie............................................................................................")
print(movie.head())
print("rating............................................................................................")
print(rating.head())
print("tag............................................................................................")
print(tag.head())

genome_scores............................................................................................
   movieId  tagId  relevance
0        1      1    0.02500
1        1      2    0.02500
2        1      3    0.05775
3        1      4    0.09675
4        1      5    0.14675
genome_tags............................................................................................
   tagId           tag
0      1           007
1      2  007 (series)
2      3  18th century
3      4         1920s
4      5         1930s
link............................................................................................
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0
movie............................................................................................
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2     

In [15]:
one = pd.merge(genome_scores,movie,on = ["movieId"])
one.head()

,movieId,tagId,relevance,title,genres
0,1,1,0.02500,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,0.02500,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,3,0.05775,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,1,4,0.09675,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,1,5,0.14675,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [18]:
one.shape

(11709768, 5)

In [91]:
yr = [i.strip(")").strip('"').strip("(").strip(" ")[-5:] for i in one.title]

In [92]:
yr = [i.strip(")").strip("(")for i in yr]
from collections import defaultdict
word_counts = defaultdict(int)
for word in yr:
    word_counts[word] += 1
word_counts

defaultdict(int,
            {'1902': 1128,
             '1915': 1128,
             '1916': 1128,
             '1917': 1128,
             '1918': 1128,
             '1919': 2256,
             '1920': 2256,
             '1921': 1128,
             '1922': 3384,
             '1923': 5640,
             '1924': 5640,
             '1925': 10152,
             '1926': 5640,
             '1927': 10152,
             '1928': 6768,
             '1929': 6768,
             '1930': 6768,
             '1931': 15792,
             '1932': 14664,
             '1933': 18048,
             '1934': 14664,
             '1935': 11280,
             '1936': 20304,
             '1937': 23688,
             '1938': 18048,
             '1939': 38352,
             '1940': 33840,
             '1941': 22560,
             '1942': 28200,
             '1943': 23688,
             '1944': 25944,
             '1945': 25944,
             '1946': 28200,
             '1947': 32712,
             '1948': 37224,
             '1949

In [93]:
odd = [i for i in one.title if ("lon 5") in i[-10:]]
word_counts = defaultdict(int)
for word in odd:
    word_counts[word] += 1
word_counts

defaultdict(int, {'Babylon 5': 1128})

In [95]:
x = []
for i in range(len(yr)):
    if(yr[i]=="lon 5"):
        yr[i] = "1994"
word_counts = defaultdict(int)
for word in yr:
    word_counts[word] += 1
word_counts

defaultdict(int,
            {'1902': 1128,
             '1915': 1128,
             '1916': 1128,
             '1917': 1128,
             '1918': 1128,
             '1919': 2256,
             '1920': 2256,
             '1921': 1128,
             '1922': 3384,
             '1923': 5640,
             '1924': 5640,
             '1925': 10152,
             '1926': 5640,
             '1927': 10152,
             '1928': 6768,
             '1929': 6768,
             '1930': 6768,
             '1931': 15792,
             '1932': 14664,
             '1933': 18048,
             '1934': 14664,
             '1935': 11280,
             '1936': 20304,
             '1937': 23688,
             '1938': 18048,
             '1939': 38352,
             '1940': 33840,
             '1941': 22560,
             '1942': 28200,
             '1943': 23688,
             '1944': 25944,
             '1945': 25944,
             '1946': 28200,
             '1947': 32712,
             '1948': 37224,
             '1949

In [96]:
yr= [int(i) for i in yr]
len(yr)

11709768

In [98]:
one["release"] = yr
one.head()

,movieId,tagId,relevance,title,genres,release
0,1,1,0.02500,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,2,0.02500,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,1,3,0.05775,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
3,1,4,0.09675,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
4,1,5,0.14675,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995


In [100]:
one.to_csv("/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/one.csv")

In [101]:
one = pd.read_csv("/media/radial/Work/Work/Work on datasets/movielens-20m-dataset*/one.csv")
two = pd.merge(one,rating,on = ["movieId"])
two.head()

MemoryError: 